In [127]:
import math
import pandas as pd
import queue
import random

In [128]:
random.seed(200)

mushroom = pd.read_csv('data/agaricus-lepiota.data')
mushroom_before_removing = mushroom

# check for missing values that are ? in the dataset
missing_values = mushroom.isin(['?']).sum()

# drop the missing values
mushroom = mushroom.replace('?', math.nan)
mushroom = mushroom.dropna()
mushroom = mushroom.reset_index(drop=True)
print(mushroom)
mushroom.loc[3984]

mushroom_training = mushroom

# using unremoved data points as a training dataset(bad idea but using for now)
mushroom_testing = mushroom_before_removing
print(mushroom_testing.loc[4023])

print(len(mushroom))
print(mushroom)
features = mushroom.columns[1:]
print(features)

variable_info = {}

for feature in features:
    variable_info[feature] = mushroom[feature].unique()

print(variable_info)

     class  cap-shape  cap-surface  cap-color  bruises  odor  gill-attachment  \
0        p          x            s          n        t     p                f   
1        e          x            s          y        t     a                f   
2        e          b            s          w        t     l                f   
3        p          x            y          w        t     p                f   
4        e          x            s          g        f     n                f   
...    ...        ...          ...        ...      ...   ...              ...   
5639     e          b            y          n        f     n                f   
5640     e          x            y          n        f     n                f   
5641     e          x            y          g        t     n                f   
5642     p          x            y          c        f     m                f   
5643     p          f            y          c        f     m                a   

      gill-spacing  gill-si

In [129]:
# entropy calculation
def entropy(num_pos, num_neg):
    if num_pos == 0 or num_neg == 0:
        return 0
    p = num_pos / (num_pos + num_neg)
    n = num_neg / (num_pos + num_neg)
    return -(p * math.log2(p) + n * math.log2(n))

# printing binary trees, with child node names no and yes
def printTree(node, level=0):
    if node != None:
        printTree(node.yes, level+1)
        print('     '* 4 * level + '-> ' + str(node))
        printTree(node.no, level+1)

# printing non-binary trees
def printNonBinaryTree(node, level=0, split_type=None):
    if node != None:
        keys = list(node.split.keys())
        num_of_values = len(keys)
        for i in range(int(num_of_values/2)):
            printNonBinaryTree(node.split[keys[i]], level+1, keys[i])
        if split_type == None:
            print('     '* 4 * level + '-> ' + str(node))
        else:
            print('     '* 4 * level + str(split_type) + ' -> ' + str(node))
        for i in range(int(num_of_values - (num_of_values/2))):
            printNonBinaryTree(node.split[keys[i+int(num_of_values/2)]], level+1, keys[i+int(num_of_values/2)])


In [130]:
# For binary tree approach

class binary_tree_node():
    def __init__(self, feature, value, no, yes):
        self.feature = feature
        self.value = value
        self.no = no
        self.yes = yes

    def add_no(self, no):
        self.no = no
    
    def add_yes(self, yes):
        self.yes = yes

    def __str__(self):
        if self.value == None:
            return f"{self.feature}"
        return f"{self.feature},{self.value}"

class DecisionTreeBinary():

    root = None

    @staticmethod
    def information_gain(entropy_prev, expected_entropy):
        return entropy_prev - expected_entropy

    @staticmethod
    def expected_entropy(num_in_feature, num_pos_in_feature, num_neg_in_feature, num_not_in_feature, num_pos_not_in_feature, num_neg_not_in_feature, total):
        return (num_in_feature/total) * entropy(num_pos_in_feature, num_neg_in_feature) + (num_not_in_feature/total) * entropy(num_pos_not_in_feature, num_neg_not_in_feature)


    def greedy_recrusive_splitting(self, data, remaining_features):
        Queue = queue.Queue()

        split_type = 'root'
        self.root = binary_tree_node(None, None, None, None)
        current_node = None

        Queue.put((data, split_type, None))
        
        while Queue.empty() == False:
            data, split_type, current_node = Queue.get()
            if len(data) == 0 or len(remaining_features) == 0:
                print('No data')
                continue
            max_gain = 0
            best_feature = None
            best_split = None
            total = len(data)
            total_pos = len(data[data['class'] == 'e'])
            total_neg = len(data[data['class'] == 'p'])
            # print(total_pos, total_neg)
            entropy_prev = entropy(total_pos, total_neg)
            # print(entropy_prev)
            if entropy_prev == 0:
                if total_pos > total_neg:
                    if split_type == 'yes':
                        current_node.add_yes(binary_tree_node('Edible', None, None, None))
                    else:
                        current_node.add_no(binary_tree_node('Edible', None, None, None))
                else:
                    if split_type == 'yes':
                        current_node.add_yes(binary_tree_node('Poisonous', None, None, None))
                    else:
                        current_node.add_no(binary_tree_node('Poisonous', None, None, None))
                print('Entropy is 0')
                continue

            for feature in remaining_features:
                for value in variable_info[feature]:
                    num_in_feature = len(data[data[feature] == value])
                    num_not_in_feature = len(data[data[feature] != value])
                    num_pos_in_feature = len(data[(data[feature] == value) & (data['class'] == 'e')])
                    num_neg_in_feature = len(data[(data[feature] == value) & (data['class'] == 'p')])
                    num_pos_not_in_feature = len(data[(data[feature] != value) & (data['class'] == 'e')])
                    num_neg_not_in_feature = len(data[(data[feature] != value) & (data['class'] == 'p')])
                    # print(num_in_feature, num_not_in_feature, feature, value, (f"{num_pos_in_feature}/{total_pos}"),num_pos_in_feature/total_pos, (f"{num_neg_in_feature}/{total_neg}"), num_neg_in_feature/total_neg)
                    expected_ent = DecisionTreeBinary.expected_entropy(num_in_feature, num_pos_in_feature, num_neg_in_feature, num_not_in_feature, num_pos_not_in_feature, num_neg_not_in_feature, total)
                    # print(expected_ent)
                    gain = DecisionTreeBinary.information_gain(entropy_prev, expected_ent)
                    # print(gain)
                    if gain > max_gain:
                        max_gain = gain
                        best_feature = feature
                        best_split = value
            print(best_feature, best_split, max_gain)
            remaining_features.remove(best_feature)
            if split_type == 'root':
                self.root = binary_tree_node(best_feature, best_split, None, None)
                current_node = self.root
            elif split_type == 'yes':
                current_node.add_yes(binary_tree_node(best_feature, best_split, None, None))
                current_node = current_node.yes
            else:
                current_node.add_no(binary_tree_node(best_feature, best_split, None, None))
                current_node = current_node.no
            
            Queue.put(((data[data[best_feature] == best_split]), 'yes', current_node))
            Queue.put(((data[data[best_feature] != best_split]), 'no', current_node))


DTB = DecisionTreeBinary()
DTB.greedy_recrusive_splitting(mushroom, list(features))

printTree(DTB.root)


 odor f 0.5375032509686903
Entropy is 0
 gill-size n 0.25641229568842117
 population s 0.24386780516394813
 ring-number o 0.16382151619508317
Entropy is 0
 stalk-shape e 0.2364527976600279
Entropy is 0
 habitat p 0.6212740427762621
 bruises t 0.3207731850041877
Entropy is 0
Entropy is 0
 stalk-surface-above-ring y 0.362051251733998
Entropy is 0
 stalk-root e 0.42969638209452654
Entropy is 0
Entropy is 0
Entropy is 0
 stalk-color-below-ring n 0.8997437586982626
Entropy is 0
Entropy is 0
                    -> Poisonous
->  odor,f
                                                            -> Poisonous
                                        ->  population,s
                                                                                                    -> Poisonous
                                                                                ->  bruises,t
                                                                                                                        -> Edibl

In [131]:
# For binary tree approach with depth

class binary_tree_node():
    def __init__(self, feature, value, no, yes):
        self.feature = feature
        self.value = value
        self.no = no
        self.yes = yes

    def add_no(self, no):
        self.no = no
    
    def add_yes(self, yes):
        self.yes = yes

    def __str__(self):
        if self.value == None:
            return f"{self.feature}"
        return f"{self.feature},{self.value}"

class DecisionTreeBinary():

    def __init__(self, training_data_set, testing_data_set, traget_feature, pos='p', neg='n'):
        self.root = None
        self.data = training_data_set
        self.test_data = testing_data_set
        self.data_features = training_data_set.columns[1:]
        self.feature_values = {}
        for feature in self.data_features:
            self.feature_values[feature] = self.data[feature].unique()
        self.target_feature=traget_feature
        self.feat_is_positive = pos
        self.feat_is_negative = neg

    @staticmethod
    def information_gain(entropy_prev, expected_entropy):
        return entropy_prev - expected_entropy

    @staticmethod
    def expected_entropy(num_in_feature, num_pos_in_feature, num_neg_in_feature, num_not_in_feature, num_pos_not_in_feature, num_neg_not_in_feature, total):
        return (num_in_feature/total) * entropy(num_pos_in_feature, num_neg_in_feature) + (num_not_in_feature/total) * entropy(num_pos_not_in_feature, num_neg_not_in_feature)


    def train(self, max_depth=10):
        remaining_features = list(self.data_features)
        Queue = queue.Queue()

        split_type = 'root'
        self.root = binary_tree_node(None, None, None, None)
        current_node = None
        depth = 0

        Queue.put((self.data, split_type, None, depth))

        while Queue.empty() == False:
            data, split_type, current_node, depth = Queue.get()
            if depth > max_depth:
                continue
            if len(data) == 0 or len(remaining_features) == 0:
                continue
            max_gain = 0
            best_feature = None
            best_split = None
            total = len(data)
            total_pos = len(data[data[self.target_feature] == self.feat_is_positive])
            total_neg = len(data[data[self.target_feature] == self.feat_is_negative])

            entropy_prev = entropy(total_pos, total_neg)

            if entropy_prev == 0:
                if total_pos > total_neg:
                    if split_type == 'yes':
                        current_node.add_yes(binary_tree_node(self.feat_is_positive, None, None, None))
                    else:
                        current_node.add_no(binary_tree_node(self.feat_is_positive, None, None, None))
                else:
                    if split_type == 'yes':
                        current_node.add_yes(binary_tree_node(self.feat_is_negative, None, None, None))
                    else:
                        current_node.add_no(binary_tree_node(self.feat_is_negative, None, None, None))
                print('Entropy is 0')
                continue

            for feature in remaining_features:
                for value in self.feature_values[feature]:

                    num_in_feature = len(data[data[feature] == value])
                    num_not_in_feature = len(data[data[feature] != value])
                    num_pos_in_feature = len(data[(data[feature] == value) & (data[self.target_feature] == self.feat_is_positive)])
                    num_neg_in_feature = len(data[(data[feature] == value) & (data[self.target_feature] == self.feat_is_negative)])
                    num_pos_not_in_feature = len(data[(data[feature] != value) & (data[self.target_feature] == self.feat_is_positive)])
                    num_neg_not_in_feature = len(data[(data[feature] != value) & (data[self.target_feature] == self.feat_is_negative)])

                    expected_ent = DecisionTreeBinary.expected_entropy(num_in_feature, num_pos_in_feature, num_neg_in_feature, num_not_in_feature, num_pos_not_in_feature, num_neg_not_in_feature, total)

                    gain = DecisionTreeBinary.information_gain(entropy_prev, expected_ent)

                    if gain > max_gain:
                        max_gain = gain
                        best_feature = feature
                        best_split = value

            print(best_feature, best_split, max_gain)
            remaining_features.remove(best_feature)

            if split_type == 'root':
                self.root = binary_tree_node(best_feature, best_split, None, None)
                current_node = self.root
            elif split_type == 'yes':
                current_node.add_yes(binary_tree_node(best_feature, best_split, None, None))
                current_node = current_node.yes
            else:
                current_node.add_no(binary_tree_node(best_feature, best_split, None, None))
                current_node = current_node.no
            
            Queue.put(((data[data[best_feature] == best_split]), 'yes', current_node, depth+1))
            Queue.put(((data[data[best_feature] != best_split]), 'no', current_node, depth+1))

    def test_accuracy(self):
        pass

    def test_accuracy_at_depth(self):
        pass
        

DTB = DecisionTreeBinary(mushroom_training, mushroom_testing, traget_feature='class', pos='e', neg='p')
DTB.train(max_depth=2)

printTree(DTB.root)

 odor f 0.5375032509686903
Entropy is 0
 gill-size n 0.25641229568842117
 population s 0.24386780516394813
 ring-number o 0.16382151619508317
                    -> p
->  odor,f
                                        ->  population,s
                    ->  gill-size,n
                                        ->  ring-number,o


In [153]:
# For non-binary tree approach

class Decision_tree_node():
    def __init__(self, feature):
        self.feature = feature
        self.split = {}

    def add(self, split_name, node):
        self.split[split_name] = node

    def __str__(self):
        return f"{self.feature}"

class DecisionTree():

    def __init__(self, training_data_set, testing_data_set, traget_feature, pos='p', neg='n'):
        self.root = None
        self.data = training_data_set
        self.test_data = testing_data_set
        self.data_features = training_data_set.columns[1:]
        self.feature_values = {}
        for feature in self.data_features:
            self.feature_values[feature] = self.data[feature].unique()
        self.target_feature=traget_feature
        self.feat_is_positive = pos
        self.feat_is_negative = neg

    @staticmethod
    def information_gain(entropy_prev, expected_entropy):
        return entropy_prev - expected_entropy

    @staticmethod
    def expected_entropy(num_in_feature, num_pos_in_feature, num_neg_in_feature, num_not_in_feature, num_pos_not_in_feature, num_neg_not_in_feature, total):
        return (num_in_feature/total) * entropy(num_pos_in_feature, num_neg_in_feature) + (num_not_in_feature/total) * entropy(num_pos_not_in_feature, num_neg_not_in_feature)


    def train(self, max_depth=10, do_output=False):
        print(f"Training Decision Tree to depth of {max_depth} -->")
        remaining_features = list(self.data_features)
        Queue = queue.Queue()

        split_type = 'root'
        self.root = Decision_tree_node(None)
        current_node = None
        depth = 0

        Queue.put((self.data, split_type, None, depth))

        while Queue.empty() == False:
            data, split_type, current_node, depth = Queue.get()
            if do_output:
                print(f'depth: {depth}', end= ":  -> ")
            if depth > max_depth:
                if do_output:
                    print(" reached desired depth")
                continue
            if len(data) == 0 or len(remaining_features) == 0:
                if do_output:
                    print(" classified everything")
                continue
            max_gain = 0
            best_feature = None
            best_split = None
            total = len(data)
            total_pos = len(data[data[self.target_feature] == self.feat_is_positive])
            total_neg = len(data[data[self.target_feature] == self.feat_is_negative])

            entropy_prev = entropy(total_pos, total_neg)

            if entropy_prev == 0:
                if total_pos > total_neg:
                    current_node.add(split_type, Decision_tree_node(self.feat_is_positive))
                else:
                    current_node.add(split_type, Decision_tree_node(self.feat_is_negative))
                if do_output:
                    print(' Entropy is 0')
                continue

            for feature in remaining_features:
                for value in self.feature_values[feature]:

                    num_in_feature = len(data[data[feature] == value])
                    num_not_in_feature = len(data[data[feature] != value])
                    num_pos_in_feature = len(data[(data[feature] == value) & (data[self.target_feature] == self.feat_is_positive)])
                    num_neg_in_feature = len(data[(data[feature] == value) & (data[self.target_feature] == self.feat_is_negative)])
                    num_pos_not_in_feature = len(data[(data[feature] != value) & (data[self.target_feature] == self.feat_is_positive)])
                    num_neg_not_in_feature = len(data[(data[feature] != value) & (data[self.target_feature] == self.feat_is_negative)])

                    expected_ent = DecisionTreeBinary.expected_entropy(num_in_feature, num_pos_in_feature, num_neg_in_feature, num_not_in_feature, num_pos_not_in_feature, num_neg_not_in_feature, total)

                    gain = DecisionTreeBinary.information_gain(entropy_prev, expected_ent)

                    if gain > max_gain:
                        max_gain = gain
                        best_feature = feature

            if split_type == 'root':
                self.root = Decision_tree_node(best_feature)
                current_node = self.root
            else:
                current_node.add(split_type, Decision_tree_node(best_feature))
                current_node = current_node.split[split_type]
            
            for value in self.feature_values[best_feature]:
                Queue.put(((data[data[best_feature] == value]), value, current_node, depth+1))

            if do_output:
                print(best_feature, max_gain)
            remaining_features.remove(best_feature)

        printNonBinaryTree(DT.root)

    def classify_item(self, item):
        current_node = self.root
        while current_node != None:
            feature = current_node.feature
            value = item[feature]
            if value not in list(current_node.split.keys()):
                break
            current_node = current_node.split[value]

            if current_node.feature == self.feat_is_positive:
                return self.feat_is_positive
            elif current_node.feature == self.feat_is_negative:
                return self.feat_is_negative

        # if the tree couldn't identify the item randomly select positive or negative
        # rand_num = random.randint(0,1)
        # if rand_num == 0:
        #     return self.feat_is_negative
        # else:
        #     return self.feat_is_positive

        # if the tree couldn't identify the item return 'does not know' 
        return 'dnk'


    def test(self):
        predictions=[]
        test_data = self.test_data.drop("class", axis=1)
        for i in range(len(self.test_data)):
            item = test_data.loc[i]
            predictions.append(self.classify_item(item))
        return predictions

    def test_accuracy(self):
        test_answers = self.test_data.loc[:, ["class"]]
        total = len(test_answers)
        correct = 0
        test_predictions = self.test()
        for i in range(total):
            if test_answers.iloc[i,0] == test_predictions[i]:
                correct += 1
        return f'{correct}/{total} -> {(correct/total)*100:.2f}%'

    def test_accuracy_at_depth(self, depth):
        self.train(max_depth=depth)
        print(f'DT depth {depth} --> Training accuracy: {self.test_accuracy()}')
        

DT = DecisionTree(mushroom_training, mushroom_testing, traget_feature='class', pos='e', neg='p')
DT.test_accuracy_at_depth(1)
DT.test_accuracy_at_depth(2)
DT.test_accuracy_at_depth(3)
DT.test_accuracy_at_depth(4)



Training Decision Tree to depth of 1 -->
                    p -> p
                    a -> e
                    l -> e
->  odor
                    n ->  spore-print-color
                    f -> p
                    c -> p
DT depth 1 --> Training accuracy: 3444/8124 -> 42.39%
Training Decision Tree to depth of 2 -->
                    p -> p
                    a -> e
                    l -> e
->  odor
                                        k -> e
                                        n -> e
                    n ->  spore-print-color
                                        r -> p
                                        w ->  population
                    f -> p
                    c -> p
DT depth 2 --> Training accuracy: 6156/8124 -> 75.78%
Training Decision Tree to depth of 3 -->
                    p -> p
                    a -> e
                    l -> e
->  odor
                                        k -> e
                                        n -> e
           